### Import Required Libraries and Set Up Environment Variables

In [2]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json
import os
from datetime import datetime
## Load the NASA_API_KEY from the env file
load_dotenv()
NASA_API_KEY = os.getenv('NASA_API_KEY')



BZm27QPtXvZHYm0NS3cgQdaP4scfeRnIGeK4k2JP


### CME Data

In [3]:
# Set the base URL to NASA's DONKI API:
base_url = "https://api.nasa.gov/DONKI/"

# Set the specifier for CMEs:
CME = "CME"

# Search for CMEs published between a begin and end date
startDate = "2024-04-01"
endDate   = "2024-05-01"

# Build URL for CME
# Build the complete URL for the CME search
url = f"{base_url}{CME}?startDate={startDate}&endDate={endDate}&api_key={NASA_API_KEY}"


In [4]:
# Make a "GET" request for the CME URL and store it in a variable named cme_response
cme_response = requests.get(url)


<Response [200]>

In [6]:
# Convert the response variable to json and store it as a variable named cme_json
cme_json= cme_response.json()


In [7]:
# Preview the first result in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(cme_json[0], indent=4))

{
    "activityID": "2024-04-01T02:36:00-CME-001",
    "catalog": "M2M_CATALOG",
    "startTime": "2024-04-01T02:36Z",
    "instruments": [
        {
            "displayName": "SOHO: LASCO/C2"
        },
        {
            "displayName": "SOHO: LASCO/C3"
        },
        {
            "displayName": "STEREO A: SECCHI/COR2"
        }
    ],
    "sourceLocation": "N12W68",
    "activeRegionNum": 13625,
    "note": "This CME is visible to the northwest in SOHO LASCO C2/C3 and STEREO A COR2 imagery. The source is an eruption near Active Region 13625 starting around 2024-04-01T01:30Z as seen in SDO AIA 131, 171, 193, 304 imagery. Faint field line movement is visible near the NW limb at this time in SDO AIA 171 and 193 as well. The eruption appears to deflect to the north.",
    "submissionTime": "2024-04-01T12:49Z",
    "versionId": 1,
    "link": "https://webtools.ccmc.gsfc.nasa.gov/DONKI/view/CME/29876/-1",
    "cmeAnalyses": [
        {
            "isMostAccurate": true,
         

In [8]:
# Convert cme_json to a Pandas DataFrame 
df = pd.DataFrame(cme_json)

# Keep only the columns: activityID, startTime, linkedEvents
df = df[['activityID', 'startTime', 'linkedEvents']]
print(df.head())

                    activityID          startTime linkedEvents
0  2024-04-01T02:36:00-CME-001  2024-04-01T02:36Z         None
1  2024-04-01T03:48:00-CME-001  2024-04-01T03:48Z         None
2  2024-04-01T08:25:00-CME-001  2024-04-01T08:25Z         None
3  2024-04-01T19:24:00-CME-001  2024-04-01T19:24Z         None
4  2024-04-01T20:36:00-CME-001  2024-04-01T20:36Z         None


In [9]:
# Notice that the linkedEvents column allows us to identify the corresponding GST
# Remove rows with missing 'linkedEvents' since we won't be able to assign these to GSTs
df_cleaned = df.dropna(subset=['linkedEvents'])
df_cleaned.head()

,activityID,startTime,linkedEvents
43,2024-04-11T07:00:00-CME-001,2024-04-11T07:00Z,[{'activityID': '2024-04-15T17:27:00-IPS-001'}]
46,2024-04-11T17:12:00-CME-001,2024-04-11T17:12Z,[{'activityID': '2024-04-11T16:52:00-FLR-001'}]
48,2024-04-12T02:00:00-CME-001,2024-04-12T02:00Z,[{'activityID': '2024-04-16T09:20:00-IPS-001'}]
52,2024-04-12T18:12:00-CME-001,2024-04-12T18:12Z,[{'activityID': '2024-04-12T17:45:00-FLR-001'}]
53,2024-04-13T05:12:00-CME-001,2024-04-13T05:12Z,[{'activityID': '2024-04-13T04:58:00-FLR-001'}]


In [10]:
# Notice that the linkedEvents sometimes contains multiple events per row
# Write a nested for loop that iterates first over each row in the cme DataFrame (using the index)
# and then iterates over the values in 'linkedEvents' 
# and adds the elements individually to a list of dictionaries where each row is one element 

# Initialize an empty list to store the expanded rows
expanded_rows = []

# Iterate over each index in the DataFrame
for index, row in df_cleaned.iterrows():
    # Iterate over each dictionary in the list
        for event in row['linkedEvents']:
        # Append a new dictionary to the expanded_rows list for each dictionary item and corresponding 'activityID' and 'startTime' value
            expanded_rows.append({
            'activityID': row['activityID'],
            'startTime': row['startTime'],
            'linkedEvent': event
            })
# Create a new DataFrame from the expanded rows
expanded_df = pd.DataFrame(expanded_rows)

expanded_df.head()

,activityID,startTime,linkedEvent
0,2024-04-11T07:00:00-CME-001,2024-04-11T07:00Z,{'activityID': '2024-04-15T17:27:00-IPS-001'}
1,2024-04-11T17:12:00-CME-001,2024-04-11T17:12Z,{'activityID': '2024-04-11T16:52:00-FLR-001'}
2,2024-04-12T02:00:00-CME-001,2024-04-12T02:00Z,{'activityID': '2024-04-16T09:20:00-IPS-001'}
3,2024-04-12T18:12:00-CME-001,2024-04-12T18:12Z,{'activityID': '2024-04-12T17:45:00-FLR-001'}
4,2024-04-13T05:12:00-CME-001,2024-04-13T05:12Z,{'activityID': '2024-04-13T04:58:00-FLR-001'}


In [11]:
# Create a function called extract_activityID_from_dict that takes a dict as input such as in linkedEvents
# and verify below that it works as expected using one row from linkedEvents as an example
# Be sure to use a try and except block to handle errors
def extract_activityID_from_dict(event_dict):
    try:
        # Extract the activityID from the dictionary
        activity_id = event_dict.get('activityID', None)
        return activity_id
    except Exception as e:
        # Log the error or print it for debugging
        print(f"Error extracting activityID: {e}")
        return None



In [12]:
# Apply this function to each row in the 'linkedEvents' column (you can use apply() and a lambda function)
# and create a new column called 'GST_ActivityID' using loc indexer:
expanded_df['GST_ActivityID'] = expanded_df['linkedEvent'].apply(lambda x: extract_activityID_from_dict(x))

print(expanded_df.head())

                    activityID          startTime  \
0  2024-04-11T07:00:00-CME-001  2024-04-11T07:00Z   
1  2024-04-11T17:12:00-CME-001  2024-04-11T17:12Z   
2  2024-04-12T02:00:00-CME-001  2024-04-12T02:00Z   
3  2024-04-12T18:12:00-CME-001  2024-04-12T18:12Z   
4  2024-04-13T05:12:00-CME-001  2024-04-13T05:12Z   

                                     linkedEvent               GST_ActivityID  
0  {'activityID': '2024-04-15T17:27:00-IPS-001'}  2024-04-15T17:27:00-IPS-001  
1  {'activityID': '2024-04-11T16:52:00-FLR-001'}  2024-04-11T16:52:00-FLR-001  
2  {'activityID': '2024-04-16T09:20:00-IPS-001'}  2024-04-16T09:20:00-IPS-001  
3  {'activityID': '2024-04-12T17:45:00-FLR-001'}  2024-04-12T17:45:00-FLR-001  
4  {'activityID': '2024-04-13T04:58:00-FLR-001'}  2024-04-13T04:58:00-FLR-001  


In [13]:
# Remove rows with missing GST_ActivityID, since we can't assign them to GSTs:
expanded_df_cleaned = expanded_df.dropna(subset=['GST_ActivityID'])


In [14]:
# print out the datatype of each column in this DataFrame:
print(expanded_df_cleaned.dtypes)


activityID        object
startTime         object
linkedEvent       object
GST_ActivityID    object
dtype: object


In [16]:
# Convert the 'GST_ActivityID' column to string format 
expanded_df_cleaned['GST_ActivityID'] = expanded_df_cleaned['GST_ActivityID'].astype(str)

# Convert startTime to datetime format  
expanded_df_cleaned['startTime'] = pd.to_datetime(expanded_df_cleaned['startTime'])

# Rename startTime to startTime_CME and activityID to cmeID
expanded_df_cleaned.rename(columns={'startTime': 'startTime_CME', 'activityID': 'cmeID'}, inplace=True)

# Drop linkedEvents
expanded_df_cleaned.drop(columns=['linkedEvent'], inplace=True)

# Verify that all steps were executed correctly
print(expanded_df_cleaned.head())
print(expanded_df_cleaned.dtypes)

KeyError: 'startTime'

In [14]:
# We are only interested in CMEs related to GSTs so keep only rows where the GST_ActivityID column contains 'GST'
# use the method 'contains()' from the str library.  
filtered_df = expanded_df_cleaned[expanded_df_cleaned['GST_ActivityID'].str.contains('GST')]
filtered_df.head()

df_cme = filtered_df

,cmeID,startTime_CME,GST_ActivityID
21,2013-06-02T20:24:00-CME-001,2013-06-02 20:24:00+00:00,2013-06-07T03:00:00-GST-001
48,2013-09-29T22:40:00-CME-001,2013-09-29 22:40:00+00:00,2013-10-02T03:00:00-GST-001
90,2013-12-04T23:12:00-CME-001,2013-12-04 23:12:00+00:00,2013-12-08T00:00:00-GST-001
148,2014-02-16T14:15:00-CME-001,2014-02-16 14:15:00+00:00,2014-02-19T03:00:00-GST-001
151,2014-02-18T01:25:00-CME-001,2014-02-18 01:25:00+00:00,2014-02-20T03:00:00-GST-001


### GST Data

In [15]:
# Set the base URL to NASA's DONKI API:
base_url = "https://api.nasa.gov/DONKI/"

# Set the specifier for Geomagnetic Storms (GST):
GST = "GST"

# Search for GSTs between a begin and end date
startDate = "2013-05-01"
endDate   = "2024-05-01"

# Build URL for GST
url = f"{base_url}{GST}?startDate={startDate}&endDate={endDate}&api_key={api_key}"


In [16]:
# Make a "GET" request for the GST URL and store it in a variable named gst_response
gst_response = requests.get(url)


In [17]:
# Convert the response variable to json and store it as a variable named gst_json
gst_json = gst_response.json()

# Preview the first result in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(gst_json[0], indent=4))

[
    {
        "gstID": "2013-06-01T01:00:00-GST-001",
        "startTime": "2013-06-01T01:00Z",
        "allKpIndex": [
            {
                "observedTime": "2013-06-01T01:00Z",
                "kpIndex": 6.0,
                "source": "NOAA"
            }
        ],
        "linkedEvents": [
            {
                "activityID": "2013-05-31T15:45:00-HSS-001"
            }
        ],
        "link": "https://webtools.ccmc.gsfc.nasa.gov/DONKI/view/GST/326/-1",
        "submissionTime": "2013-07-15T19:26Z"
    }
]


In [18]:
# Convert gst_json to a Pandas DataFrame  
df = pd.DataFrame(gst_json)

# Keep only the columns: activityID, startTime, linkedEvents
df = df[['activityID', 'startTime', 'linkedEvents']]
df.head()

,gstID,startTime,linkedEvents
0,2013-06-01T01:00:00-GST-001,2013-06-01T01:00Z,[{'activityID': '2013-05-31T15:45:00-HSS-001'}]
1,2013-06-07T03:00:00-GST-001,2013-06-07T03:00Z,[{'activityID': '2013-06-02T20:24:00-CME-001'}]
2,2013-06-29T03:00:00-GST-001,2013-06-29T03:00Z,None
3,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,[{'activityID': '2013-09-29T22:40:00-CME-001'}...
4,2013-12-08T00:00:00-GST-001,2013-12-08T00:00Z,[{'activityID': '2013-12-04T23:12:00-CME-001'}...


In [19]:
# Notice that the linkedEvents column allows us to identify the corresponding CME
# Remove rows with missing 'linkedEvents' since we won't be able to assign these to CME
df_cleaned = df.dropna(subset=['linkedEvents'])
df_cleaned.head()

,gstID,startTime,linkedEvents
0,2013-06-01T01:00:00-GST-001,2013-06-01T01:00Z,[{'activityID': '2013-05-31T15:45:00-HSS-001'}]
1,2013-06-07T03:00:00-GST-001,2013-06-07T03:00Z,[{'activityID': '2013-06-02T20:24:00-CME-001'}]
3,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,[{'activityID': '2013-09-29T22:40:00-CME-001'}...
4,2013-12-08T00:00:00-GST-001,2013-12-08T00:00Z,[{'activityID': '2013-12-04T23:12:00-CME-001'}...
5,2014-02-19T03:00:00-GST-001,2014-02-19T03:00Z,[{'activityID': '2014-02-16T14:15:00-CME-001'}...


In [20]:
# Notice that the linkedEvents sometimes contains multiple events per row
# Use the explode method to ensure that each row is one element. Ensure to reset the index and drop missing values.
df_exploded = df_cleaned.explode('linkedEvents').reset_index(drop=True)
df_exploded = df_exploded.dropna(subset=['linkedEvents'])


In [21]:
# Apply the extract_activityID_from_dict function to each row in the 'linkedEvents' column (you can use apply() and a lambda function)
# and create a new column called 'CME_ActivityID' using loc indexer:
df_exploded['CME_ActivityID'] = df_exploded['linkedEvents'].apply(lambda x: extract_activityID_from_dict(x))

# Remove rows with missing CME_ActivityID, since we can't assign them to CMEs:
df_final = df_exploded.dropna(subset=['CME_ActivityID'])
df_final.head()

,gstID,startTime,linkedEvents,CME_ActivityID
0,2013-06-01T01:00:00-GST-001,2013-06-01T01:00Z,{'activityID': '2013-05-31T15:45:00-HSS-001'},2013-05-31T15:45:00-HSS-001
1,2013-06-07T03:00:00-GST-001,2013-06-07T03:00Z,{'activityID': '2013-06-02T20:24:00-CME-001'},2013-06-02T20:24:00-CME-001
2,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,{'activityID': '2013-09-29T22:40:00-CME-001'},2013-09-29T22:40:00-CME-001
3,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,{'activityID': '2013-10-02T01:54:00-IPS-001'},2013-10-02T01:54:00-IPS-001
4,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,{'activityID': '2013-10-02T02:47:00-MPC-001'},2013-10-02T02:47:00-MPC-001


In [22]:
# Convert the 'CME_ActivityID' column to string format 
df_final['CME_ActivityID'] = df_final['CME_ActivityID'].astype(str)

# Convert the 'gstID' column to string format 
df_final['activityID'] = df_final['activityID'].astype(str)

# Convert startTime to datetime format  
df_final['startTime'] = pd.to_datetime(df_final['startTime'])

# Rename startTime to startTime_GST 
df_final.rename(columns={'startTime': 'startTime_GST'}, inplace=True)

# Drop linkedEvents
df_final.drop(columns=['linkedEvents'], inplace=True)

# Verify that all steps were executed correctly
print(df_final.info())
print(df_final.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 217 entries, 0 to 216
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   gstID           217 non-null    string             
 1   startTime_GST   217 non-null    datetime64[ns, UTC]
 2   CME_ActivityID  217 non-null    string             
dtypes: datetime64[ns, UTC](1), string(2)
memory usage: 5.2 KB


In [23]:
# We are only interested in GSTs related to CMEs so keep only rows where the CME_ActivityID column contains 'CME'
# use the method 'contains()' from the str library.  
df_filtered = df_final[df_final['CME_ActivityID'].str.contains('CME')]
df_filtered.head()

df_gst = df_filtered

,gstID,startTime_GST,CME_ActivityID
1,2013-06-07T03:00:00-GST-001,2013-06-07 03:00:00+00:00,2013-06-02T20:24:00-CME-001
2,2013-10-02T03:00:00-GST-001,2013-10-02 03:00:00+00:00,2013-09-29T22:40:00-CME-001
5,2013-12-08T00:00:00-GST-001,2013-12-08 00:00:00+00:00,2013-12-04T23:12:00-CME-001
7,2014-02-19T03:00:00-GST-001,2014-02-19 03:00:00+00:00,2014-02-16T14:15:00-CME-001
9,2014-02-20T03:00:00-GST-001,2014-02-20 03:00:00+00:00,2014-02-18T01:25:00-CME-001


### Merge both datatsets

In [24]:
# Now merge both datasets using 'gstID' and 'CME_ActivityID' for gst and 'GST_ActivityID' and 'cmeID' for cme. Use the 'left_on' and 'right_on' specifiers.
merged_df = df_gst.merge(df_cme, left_on='CME_ActivityID', right_on='cmeID', suffixes=('_GST', '_CME'))

merged_df.head()

,gstID,startTime_GST,CME_ActivityID,cmeID,startTime_CME,GST_ActivityID
0,2013-06-07T03:00:00-GST-001,2013-06-07 03:00:00+00:00,2013-06-02T20:24:00-CME-001,2013-06-02T20:24:00-CME-001,2013-06-02 20:24:00+00:00,2013-06-07T03:00:00-GST-001
1,2013-10-02T03:00:00-GST-001,2013-10-02 03:00:00+00:00,2013-09-29T22:40:00-CME-001,2013-09-29T22:40:00-CME-001,2013-09-29 22:40:00+00:00,2013-10-02T03:00:00-GST-001
2,2013-12-08T00:00:00-GST-001,2013-12-08 00:00:00+00:00,2013-12-04T23:12:00-CME-001,2013-12-04T23:12:00-CME-001,2013-12-04 23:12:00+00:00,2013-12-08T00:00:00-GST-001
3,2014-02-19T03:00:00-GST-001,2014-02-19 03:00:00+00:00,2014-02-16T14:15:00-CME-001,2014-02-16T14:15:00-CME-001,2014-02-16 14:15:00+00:00,2014-02-19T03:00:00-GST-001
4,2014-02-20T03:00:00-GST-001,2014-02-20 03:00:00+00:00,2014-02-18T01:25:00-CME-001,2014-02-18T01:25:00-CME-001,2014-02-18 01:25:00+00:00,2014-02-20T03:00:00-GST-001


In [25]:
# Verify that the new DataFrame has the same number of rows as cme and gst
num_rows_gst = len(df_gst)
num_rows_cme = len(df_cme)
num_rows_merged = len(merged_df)

print(f"Number of rows in GST DataFrame: {num_rows_gst}")
print(f"Number of rows in CME DataFrame: {num_rows_cme}")
print(f"Number of rows in merged DataFrame: {num_rows_merged}")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67 entries, 0 to 66
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   gstID           67 non-null     string             
 1   startTime_GST   67 non-null     datetime64[ns, UTC]
 2   CME_ActivityID  67 non-null     object             
 3   cmeID           67 non-null     object             
 4   startTime_CME   67 non-null     datetime64[ns, UTC]
 5   GST_ActivityID  67 non-null     string             
dtypes: datetime64[ns, UTC](2), object(2), string(2)
memory usage: 3.3+ KB


### Computing the time it takes for a CME to cause a GST

In [26]:
# Compute the time diff between startTime_GST and startTime_CME by creating a new column called `timeDiff`.
merged_df['timeDiff'] = merged_df['startTime_GST'] - merged_df['startTime_CME']
merged_df.head()

,gstID,startTime_GST,CME_ActivityID,cmeID,startTime_CME,GST_ActivityID,timeDiff
0,2013-06-07T03:00:00-GST-001,2013-06-07 03:00:00+00:00,2013-06-02T20:24:00-CME-001,2013-06-02T20:24:00-CME-001,2013-06-02 20:24:00+00:00,2013-06-07T03:00:00-GST-001,4 days 06:36:00
1,2013-10-02T03:00:00-GST-001,2013-10-02 03:00:00+00:00,2013-09-29T22:40:00-CME-001,2013-09-29T22:40:00-CME-001,2013-09-29 22:40:00+00:00,2013-10-02T03:00:00-GST-001,2 days 04:20:00
2,2013-12-08T00:00:00-GST-001,2013-12-08 00:00:00+00:00,2013-12-04T23:12:00-CME-001,2013-12-04T23:12:00-CME-001,2013-12-04 23:12:00+00:00,2013-12-08T00:00:00-GST-001,3 days 00:48:00
3,2014-02-19T03:00:00-GST-001,2014-02-19 03:00:00+00:00,2014-02-16T14:15:00-CME-001,2014-02-16T14:15:00-CME-001,2014-02-16 14:15:00+00:00,2014-02-19T03:00:00-GST-001,2 days 12:45:00
4,2014-02-20T03:00:00-GST-001,2014-02-20 03:00:00+00:00,2014-02-18T01:25:00-CME-001,2014-02-18T01:25:00-CME-001,2014-02-18 01:25:00+00:00,2014-02-20T03:00:00-GST-001,2 days 01:35:00


In [27]:
# Use describe() to compute the mean and median time 
# that it takes for a CME to cause a GST. 
print(merged_df['timeDiff'].describe())


,timeDiff
count,67
mean,2 days 21:35:13.432835820
std,1 days 00:02:46.681279427
min,1 days 05:36:00
25%,2 days 03:12:00
50%,2 days 17:48:00
75%,3 days 12:17:00
max,6 days 03:00:00


### Exporting data in csv format

In [28]:
# Export data to CSV without the index
merged_df.to_csv('merged_data.csv', index=False)